In [1]:
from py5paisa import FivePaisaClient
from py5paisa.order import Order, OrderType, Exchange, Bo_co_order
import time
import math
import pandas as pd
import warnings
from datetime import datetime

In [2]:
warnings.filterwarnings("ignore", category=FutureWarning)

In [3]:
cred = {
    "APP_NAME": "5P56409084",
    "APP_SOURCE": "9217",
    "USER_ID": "pybeQPEbjju",
    "PASSWORD": "AESDZQGqyAa",
    "USER_KEY": "542GTEZguBbLERUXRZe26rLvfWU2X2KD",
    "ENCRYPTION_KEY": "l8sRTIsjl1JhUrz8X48XQJzKGI4k0gft",
}
client = FivePaisaClient(
    email="vasuappliedai@gmail.com", passwd="Vasudeva@1", dob="19981218", cred=cred
)
client.login()

 23:52:03 | Logged in!!


In [5]:
def get_current_expiry_strikes(symbol, start_strike, end_strike):
    temp_df = script_df["FullName"].str.contains(
        pat=symbol + " CE", regex=True)
    call_df = script_df[temp_df]
    call_df = call_df[call_df["StrikeRate"] > start_strike]
    call_df = call_df[call_df["StrikeRate"] < end_strike]
    call_df.to_csv("bank_nifty_call_expiry.csv")
    temp_df = script_df["FullName"].str.contains(
        pat=symbol + " PE", regex=True)
    put_df = script_df[temp_df]
    put_df = put_df[put_df["StrikeRate"] > start_strike]
    put_df = put_df[put_df["StrikeRate"] < end_strike]
    put_df.to_csv("bank_nifty_put_expiry.csv")
    return call_df, put_df

def map_prices_to_strike_prices(data_frame):
    temp_rows = []
    for _, row in data_frame.iterrows():
        script_code = row["Scripcode"]
        try:
            close_point = client.historical_data(
                "N", "D", script_code, "5m", given_date, given_date
            ).tail(1)
            temp_rows.append(close_point["Close"].iloc[0])
        except:
            continue
    data_frame["LTP"] = temp_rows
    return data_frame.sort_values(by=["LTP"])

def rounded_number(number):
    return (round(strikePrice / 1000, 1) * 1000).astype(int)

In [6]:
# This code should be executed on Thursday or when profit is booked
nifty_script_code = 999920000
banknifty_script_code = 999920005
given_date = datetime.today().strftime("%Y-%m-%d")
script_df = pd.read_csv("scripmaster-csv-format.csv",low_memory=False)
symbol = "BANKNIFTY 31 Mar 2022"
strikePrice = client.historical_data("N", "C", banknifty_script_code, "1m", given_date, given_date).tail(1)["Open"].iloc[0]
strikePrice = rounded_number(strikePrice)
strikePrice

35500

In [ ]:
call_df, put_df = get_current_expiry_strikes(symbol, strikePrice*0.94, strikePrice*1.06)
call_df,put_df

In [42]:
call_option_chain = map_prices_to_strike_prices(call_df)
put_option_chain = map_prices_to_strike_prices(put_df)

In [7]:
import pickle
# option_chains = (call_option_chain,put_option_chain)
# with open("option_chain.pickle","wb") as file:
#     pickle.dump(option_chains,file)

file = open("option_chain.pickle","rb")
call_option_chain,put_option_chain = pickle.load(file)

In [8]:
desired_call_price = 115
desired_put_price = 115


In [11]:
call_option_chain[call_option_chain["LTP"]<desired_call_price].tail(1)

,Exch,ExchType,Scripcode,Name,Series,Expiry,CpType,StrikeRate,WireCat,ISIN,FullName,LTP
127950,N,D,58310,BANKNIFTY 31 Mar 2022 CE 37300.00,XX,2022-03-31 14:30:00,CE,37300.0,,BANKNIFTY,BANKNIFTY 31 Mar 2022 CE 37300.00,109.0


In [12]:
put_option_chain[put_option_chain["LTP"]<desired_put_price].tail(1)

,Exch,ExchType,Scripcode,Name,Series,Expiry,CpType,StrikeRate,WireCat,ISIN,FullName,LTP
127885,N,D,58209,BANKNIFTY 31 Mar 2022 PE 33800.00,XX,2022-03-31 14:30:00,PE,33800.0,,BANKNIFTY,BANKNIFTY 31 Mar 2022 PE 33800.00,108.65


In [9]:
call_script_code = call_option_chain[call_option_chain["LTP"]<desired_call_price].tail(1)["Scripcode"].iloc[0]
put_script_code = put_option_chain[put_option_chain["LTP"]<desired_put_price].tail(1)["Scripcode"].iloc[0]
call_script_code,put_script_code

(58310, 58209)

In [10]:
entry_point_index = 345
call_points = client.historical_data('N','D',call_script_code,'1m',given_date,given_date)["Close"].to_list()[entry_point_index:]
put_points = client.historical_data('N','D',put_script_code,'1m',given_date,given_date)["Close"].to_list()[entry_point_index:]
for i in range(min(len(call_points),len(put_points))):
    if(call_points[i]*0.5 > put_points[i]):
        print("Change stoploss here")
    elif(call_points[i] < put_points[i]*0.5):
        print("Change stoploss here")

In [13]:
def day_close_check(call_script_code,put_script_code):
    call_point = client.historical_data('N','D',call_script_code,'1m',given_date,given_date).tail(1)["Close"].iloc[0]
    put_point = client.historical_data('N','D',put_script_code,'1m',given_date,given_date).tail(1)["Close"].iloc[0]
    if(call_point < put_point*0.75):
        print("Buy CE and sell CE near to put strike price")
        adjust_strangle("CE",put_point*0.8)
    if(put_point < call_point*0.75):
        print("Buy PE and sell PE near to call strike price")




In [ ]:
def adjust_strangle(option_type,desired_price):
    call_df, put_df = get_current_expiry_strikes(symbol, strikePrice*0.94, strikePrice*1.06)
    call_option_chain = map_prices_to_strike_prices(call_df)
    put_option_chain = map_prices_to_strike_prices(put_df)
    if option_type == "CE":
        call_script_code = call_option_chain[call_option_chain["LTP"]<desired_price].tail(1)["Scripcode"].iloc[0]
        call_sell = call_option_chain[call_option_chain["LTP"]<desired_price].tail(1)["LTP"].iloc[0]
        call_symbol = call_option_chain[call_option_chain["LTP"]<desired_price].tail(1)["Name"].iloc[0]
        print("Sold call sell at and Symbol",call_sell,call_symbol)
    if option_type == "PE":
        put_script_code = put_option_chain[put_option_chain["LTP"]<desired_price].tail(1)["Scripcode"].iloc[0]
        put_sell = put_option_chain[put_option_chain["LTP"]<desired_price].tail(1)["LTP"].iloc[0]
        put_symbol = put_option_chain[put_option_chain["LTP"]<desired_price].tail(1)["Name"].iloc[0]
        print("Sold put sell at and Symbol",put_sell,put_symbol)

    
    


    

In [14]:
client.get_tradebook()

 00:19:20 | 'body'
